In [1]:

"""
This file can be used to generate inflfow and outflow at BOTH cluster-level (H=60 ,d = 0.5) and station-level (H=10 ,d = 0.01)

cluster parameters:
columns=["start_cluster"] #for cluster demand
H,d = 60,0.5 

station parameters:
columns=["start_station"] #for station demand
H,d = 10,0.01 

"""

'\nThis file can be used to generate inflfow and outflow at BOTH cluster-level (H=60 ,d = 0.5) and station-level (H=10 ,d = 0.01)\n\ncluster parameters:\ncolumns=["start_cluster"] #for cluster demand\nH,d = 60,0.5 \n\nstation parameters:\ncolumns=["start_station"] #for station demand\nH,d = 10,0.01 \n\n'

In [2]:
import datetime
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from haversine import haversine_vector, Unit
import os

(CVXPY) May 24 07:34:13 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


In [3]:
def temporal_clustering(H,demand):
    
    def str_to_dt(dt_str):
        return datetime.datetime.strptime(dt_str, '%H:%M:%S').time()

    
    # temporal clustering
    demand.insert(3,'start_timeslot', demand['start_time'].apply(lambda x:int(np.floor((60*str_to_dt(x).hour + str_to_dt(x).minute)/H))))
    demand.insert(6,'end_timeslot', demand['end_time'].apply(lambda x:int(np.floor((60*str_to_dt(x).hour + str_to_dt(x).minute)/H))))


In [4]:
def spatial_clustering(d,demand):
    os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\station_names_n_cluster_center_id')
    stations = pd.read_csv('stations.csv',index_col=0)
    
    stn_coords = list(zip(stations.y, stations.x))

    D = haversine_vector(stn_coords, stn_coords, Unit.KILOMETERS, comb=True)
    R = 1*(D < d) # 1 km threshold
    
    # ilp to select cluster centers
    x = cp.Variable((len(stations),),boolean=True)
    prob = cp.Problem(cp.Minimize(cp.sum(x)), [x.T@R>=1])
    prob.solve()
    
    # map each station to a cluster
    stations['cc'] = x.value
    cluster_centers = stations.loc[stations.cc==1]
    cc_coords = list(zip(cluster_centers.y, cluster_centers.x))
    stations['cc_id'] = np.argmin(haversine_vector(cc_coords, stn_coords, Unit.KILOMETERS, comb=True), axis=1)
    
    station_to_cluster_map = dict(zip(stations.index, stations.cc_id))
    
    # spatial clustering
    demand['start_cluster'] = demand['start_station'].apply(lambda x: station_to_cluster_map[x])
    demand['end_cluster'] = demand['end_station'].apply(lambda x: station_to_cluster_map[x])
    
    return station_to_cluster_map, stations

In [5]:
if __name__ == "__main__":
    os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\\raw_data')
    demand = pd.read_csv('transaction_logs.csv',index_col=0)

## start
# DATAFRAME T cluster level demand

    H,d = 60,0.5 # H:Timeslot duration in minutes, d = radius for clustering in km
    #temporal clustering
    temporal_clustering(H,demand)
    
    #spatial clustering
    station_to_cluster_map, stations = spatial_clustering(d,demand)
    print(station_to_cluster_map)
    #print(stations['region'])
    os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\station_names_n_cluster_center_id')
    stations.to_csv('station_cc_id_'+str(H)+'_'+str(d)+'.csv', index=False)

    #aggregated demand table
    T = pd.pivot_table(demand, values = 'car', index=["start_date","start_timeslot"], columns=["start_cluster"], aggfunc=lambda x: len(x.unique()))
    IN = pd.pivot_table(demand, values = 'car', index=["end_date","end_timeslot"], columns=["end_cluster"], aggfunc=lambda x: len(x.unique()))
    IN.fillna(0, inplace=True)
    T.fillna(0, inplace=True)
    
    # missing_clusters = [x for x in range(T.columns[-1]) if x not in T.columns]
    # missing_clusters.sort()

    # for ms in missing_clusters:
    #     T.insert(ms, ms, 0)
    """
    Ensure that the index of T and IN are of the same length and aligned

    """
    start_idx = max(IN.index[0],T.index[0])
    end_idx = min(IN.index[-1],T.index[-1])
    T = T.loc[start_idx:end_idx]
    IN = IN.loc[start_idx:end_idx]

    print(T)
    os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\cluster_level\outflow_data')
    T.to_csv('cluster_outflow_'+str(start_idx)+'_to_'+str(end_idx)+'.csv', index=False)

    # missing_clusters = [x for x in range(IN.columns[-1]) if x not in IN.columns]
    # missing_clusters.sort()
    
    # for ms in missing_clusters:
    #     IN.insert(ms, ms, 0)

    print(IN)
    os.chdir('C:\Work\WORK_PACKAGE\Demand_forecasting\github\DeepAR-pytorch\My_model\Rental_Fleet_Demand_Forecast\data\cluster_level\inflow_data')
    IN.to_csv('cluster_inflow_'+str(start_idx)+'_to_'+str(end_idx)+'.csv', index=False)
    
## stop

    #raw_data = T.to_numpy()


{34: 2, 20: 0, 298: 1, 140: 8, 329: 1, 285: 2, 272: 2, 286: 3, 203: 4, 294: 5, 328: 8, 195: 5, 46: 6, 119: 7, 355: 8, 77: 8, 93: 10, 192: 9, 35: 10, 348: 9, 163: 10, 310: 11, 202: 11, 318: 12, 245: 12, 206: 13, 352: 14, 190: 11, 178: 15, 129: 16, 166: 16, 345: 16, 370: 17, 29: 18, 18: 19, 331: 20, 218: 20, 251: 20, 56: 21, 249: 22, 149: 23, 154: 24, 326: 24, 210: 25, 191: 24, 32: 27, 239: 25, 388: 26, 278: 27, 306: 27, 186: 28, 170: 26, 94: 31, 139: 29, 141: 32, 377: 30, 116: 31, 209: 35, 376: 46, 151: 31, 385: 34, 132: 33, 78: 32, 366: 33, 150: 32, 320: 34, 9: 34, 155: 35, 389: 36, 288: 35, 58: 35, 323: 38, 162: 37, 43: 38, 330: 38, 325: 39, 118: 40, 110: 37, 15: 39, 341: 39, 307: 41, 84: 38, 304: 40, 138: 42, 296: 43, 147: 44, 126: 45, 228: 49, 263: 56, 113: 46, 243: 47, 81: 49, 74: 48, 257: 49, 368: 56, 82: 50, 5: 51, 98: 52, 47: 52, 17: 53, 37: 54, 168: 55, 41: 46, 40: 56, 16: 107, 28: 54, 26: 57, 30: 56, 344: 58, 73: 59, 64: 60, 181: 60, 128: 61, 378: 62, 281: 62, 271: 64, 275: 64